In [1]:
#### Dependencies

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [2]:
# Create a list of paths to our raw data files

DATA_DIR = 'data/processed/'

### List of files to be read

files = os.listdir(DATA_DIR)
#print(files)

full_file_list = []

for file in files:
    full_file_list.append(DATA_DIR + file)
print(full_file_list)


['data/processed/april_2017.csv', 'data/processed/data_load_clean.json', 'data/processed/jan_2018.csv', 'data/processed/may_2018.csv', 'data/processed/oct_2017.csv', 'data/processed/sept_2018.csv', 'data/processed/strain_name_formal_match.csv']


In [3]:
### Read The CSVs as dataframes

jan_2018 = pd.read_csv('data/processed/jan_2018.csv')
april_2017 = pd.read_csv('data/processed/april_2017.csv')
may_2018 = pd.read_csv('data/processed/may_2018.csv')
oct_2017 = pd.read_csv('data/processed/oct_2017.csv')
sept_2018 = pd.read_csv('data/processed/sept_2018.csv')

In [4]:
may_2018.head()
may_2018.columns


Index(['Unnamed: 0', 'Timestamp_x', 'plant_id', 'Trimmer', 'grams',
       'Simple_Date', 'Unnamed: 5', 'Timestamp_y', 'Name', 'Flower Time'],
      dtype='object')

In [5]:
### Merge them into a single DF indexed by timestamp (Timesstamp_x)

### Rename all the columns to be the same
jan_2018.rename(columns={'Timestamp_x':'time'}, inplace=True)
april_2017.rename(columns={'Timestamp':'time'}, inplace=True)
sept_2018.rename(columns={'Timestamp_x':'time'}, inplace=True)
oct_2017.rename(columns={'Timestamp':'time'}, inplace=True)
may_2018.rename(columns={'Timestamp_x':'time'}, inplace=True)





In [6]:


### Merge all the dataframes into a single dataframe

df = pd.concat([jan_2018, april_2017, may_2018, oct_2017, sept_2018], axis=0, sort=False)
# df.head()
# df.columns
# df.info()

### Clean up 
# # Dropp uneeded columns
# 'Unnamed: 0', 'SImple Date', 'Simple_Date'

df.drop(['Unnamed: 0', 'SImple Date', 'Simple_Date', 'Unnamed: 5'], axis=1, inplace=True)


In [7]:
# df.info()
# df.head()


### Look at the strain names

df['Name'].value_counts()

### Solving match problem
## change 'PBB' in the name column to 'Peanut Butter Breath' for better name matching

df['Name'] = df['Name'].replace('PBB', 'Peanut Butter Breath')

### Export a list of the unique strain names

list = df['Name'].unique().tolist()
len(list)

## sort list in alphabetical order

list.sort()
list

### Export the list to a csv

list_df = pd.DataFrame(list)

# list_df.to_csv('TEMP/strain_names_raw.csv', index=False)



In [8]:
### Clean up and standardize the strain names

### Read the of clean strain names

clean_names = pd.read_csv('data\processed\strain_name_formal_match.csv')

clean_names.head()
clean_names.info()

### Create a list of the clean names

clean_list = clean_names['name'].tolist()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    43 non-null     object
dtypes: object(1)
memory usage: 472.0+ bytes


In [9]:
### Use fuzzywuzzy to match the names
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### Create a list of the fuzzy matches

matches = []

for name in list:
    match = process.extractOne(name, clean_list)
    matches.append(match)

### Create a dataframe with the original name, the match name and score in different columns
## New frame just with the results
matches_df = pd.DataFrame(matches, columns=['match', 'score'])

### Merge the matches with the original list
matches_df = pd.concat([list_df, matches_df], axis=1)
## Rename the first column
matches_df.rename(columns={0:'name'}, inplace=True)

### Replace the names in the original dataframe with the match names
df['Name'] = df['Name'].replace(matches_df['name'].tolist(), matches_df['match'].tolist())

#### Clean up the Trimmer names
## names to combine
Brie = ['Brie', 'Bri']
Smitty = ['Smit', 'Smitty']
T_Pain = ['TPain', 'T Pain']
Gmoney = ['Gmoney', 'Grant']

df['Trimmer'] = df['Trimmer'].replace(Brie, 'Brie')
df['Trimmer'] = df['Trimmer'].replace(Smitty, 'Smitty')
df['Trimmer'] = df['Trimmer'].replace(T_Pain, 'T Pain')
df['Trimmer'] = df['Trimmer'].replace(Gmoney, 'Gmoney')

# matches_df.head()


In [15]:
########## Working up to here ########## Exports below to single csv file

In [16]:


df['Name'].value_counts()

### Export the dataframe to a file to use in tableau

df.to_csv('data/processed/viz.csv', index=False)